# Visualizing Text Data

The unstructured nature of text data makes it much more difficult to analyze.  Because it often takes a lot of preprocessing to visualize text data, we'll use the ChatGPT Data Analyst to help us code and visualize our datasets.


### European Restaurant Reviews

### **Query**: "Please suggest 5 different visualizations that could be used to show patterns in the text of positive and negative reviews."